# 01-grad-otsu-max.ipynb
This notebook is to begin the segmentation procedure outline in `README.md`. 
Steps performed here include:
- [Load data](#load-data)
- [(A) Smooth image](#a-smooth-image)
- [(B) The gradient magnitude of A](#b-calculate-gradient)
- [(C) Explorative method for selecting some maxima using multi-Itsu thresholding](#c-select-maxima-with-multi-otsu)

# Import packages

In [1]:
use_napari = True

In [2]:
# Standard library imports
from pathlib import Path
# Third-party imports
import imagecodecs  # dependency required for loading compressed tif images
import imageio as iio
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters, morphology, util
if use_napari:
    import napari

# Load data

In [3]:
if use_napari:
    viewer = napari.Viewer()

In [4]:
project_dir = Path('..').resolve()
print(project_dir)

C:\Users\gusb\Research\mhe-analysis


In [5]:
img_path = Path(project_dir / 'IDOX_500.tif')   
img = iio.imread(img_path)
print(f'{img.shape=}')
print(f'{img.dtype=}')

img.shape=(1004, 924)
img.dtype=dtype('uint16')


In [6]:
def plot_img(img):
    fig, ax = plt.subplots(figsize = (8, 8))
    ax.imshow(img)
    ax.axis('off')
    return fig, ax

In [7]:
if use_napari:
    viewer.add_image(img, name='raw')
else:
    fig, ax = plot_img(img)

# (A) Smooth image

In [8]:
img_smooth = filters.gaussian(img)
if use_napari:
    viewer.add_image(img_smooth, name='smoothed')
else:
    fig, ax = plot_img(img_smooth)

# (B) Calculate gradient

In [9]:
img_grad = filters.rank.gradient(img_smooth, morphology.disk(3))
if use_napari:
    viewer.add_image(img_grad, name='gradient')
else:
    fig, ax = plot_img(img_grad)

C:\Users\gusb\Python\envs\napari-env\lib\site-packages\IPython\core\interactiveshell.py:3437: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  exec(code_obj, self.user_global_ns, self.user_ns)


# (C) Select maxima with multi-Otsu

In [10]:
thresholds = filters.threshold_multiotsu(img_smooth, classes=3)
thresholds

array([0.22145649, 0.55121652])

In [11]:
def thresh_multiotsu(img, n_classes=3, show_process=False): 
    thresholds = filters.threshold_multiotsu(img_smooth, classes=n_classes)
    img_multiotsu = np.zeros_like(img)
    if show_process:
        fig, axes = plt.subplots(1, 3, figsize=(12, 5))
        ax = axes.ravel()
        ax[0].imshow(img)
        ax[1].hist(img.ravel(), bins=256)
        for i, thresh in enumerate(thresholds):
            ax[1].axvline(thresh, color='black')
            img_multiotsu[img > thresh] = (i + 1) / len(thresholds)
        ax[1].set_ylim([0, 15000])
        ax[2].imshow(img_multiotsu)
        return fig, axes
    else:
        for i, thresh in enumerate(thresholds):
            img_multiotsu[img > thresh] = (i + 1) / len(thresholds)
        return img_multiotsu

In [12]:
n_classes = 3
if use_napari:
    multiotsu = thresh_multiotsu(img_smooth, n_classes=n_classes)
    viewer.add_image(multiotsu, name=f'multi-Otsu: {n_classes=}')
else:
    fig, axes = thresh_multiotsu(
        img_smooth, n_classes=n_classes, show_process=True
    )

In [14]:
n_classes = 4
if use_napari:
    multiotsu = thresh_multiotsu(img_smooth, n_classes=n_classes)
    viewer.add_image(multiotsu, name=f'multi-Otsu: {n_classes=}')
else:
    fig, axes = thresh_multiotsu(
        img_smooth, n_classes=n_classes, show_process=True
    )